Project one is to fit the simplest model EUT to data
will use curve_fit from scipy and look into pyomo for abstract functions


CURRENT PROGRESS

model -creation/predictions are done.

Data sorting is done

Now we are fitting

We have used scipy.optimize to optimize our params
we are using get_loss_one as the objective function which finds the mean squared error averaged between the preds and actuals.

We have used nelder mead, powell, cobyla. These however have produced bad results for the top 10 participants.

We have tried longer optimization algorithms such as binhopping and shgo however they are taking over 20 minutes per participant without sign of slowing down therefore we are not going to use them.

Differential evolution seems to be working the best. We have implemetned a constraint over the optimzation to make sure that the values stay within the bounds and constrains outlined in the report Glass gave us.

The runtime seems to be around 13-15 minutes per participant.
we can try and optimize the algorithm by closing all tabs, and modifying the parameters however trying a gridsearch or parameters search for this tuning may take too long however if optimal parameters can be broadcast to all participants it might be worth it.

We are yet to try dual annealing or the DIRECT method from scipy.optimize and we haven't attempted the recursive linear fit outlined in yudelson's code.
Yudelson's greedy fit I think will take too long, however we haven't tried running it for over 10 minutes yet.

Here are the DE results for partiicpant 1 (p=0).

array**([a: 0.25019451, b: 0.26774997,l: 1.19726157, g: 0.99461636]**)
although this is vastly different than the actual params the yielded predictions are the same.

predictions for participant 7
array([0.89365145, 0.92065334, 1.38266249, 0.84074097])

8/18/23
We fit the first 4 participants using the differential evolution

it took an estimated time of 48 minutes 12 per participant.
While participants 1,3 and 4 resulted in d2a's of less than 11 percent
participant 2 had a fit that was completely off and missed the prediction on day 4.

To work on next we try and change the preciscion or how much the model is jumping by to make the search space smalleer. For example, we can make it only search by the hundreths.

Another thing we can do is we can run the fitting with low iterations and then go back and re fit the participants that had bad results until the algo converges.

One pattern that might be present, is that when alhpa and beta are low, the model might not work.

To improve preciscion, we can increase the iteration size as well as decrease the custom tolerance for the d2a error, making it so that it will only stop once it's really small.

Instructions for running:

run all the neccescary functions and all (few important ones you might miss are the d2a_error, creating soldOnly).
Make sure you scroll down to where I say first4=[ array, array ,array, array ...]
and run that since the fitting loop here needs it
make sure to initialize the fiveto8 list as empty
make sure to put the callbacks into the call_backs list
then just run the loop, it should take quite some time as I tripled the maxiter


WANDB KEY : 61cca3bef0200dc37d895eb5a336add7f668eb69


In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
NEIGHBORS_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/'

In [3]:
POLISHED_DIRECT_PATH = '/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/'

In [4]:
DIRECT_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/DIRECT fits/'  # THIS IS SPECIFIC TO MY DRIVE, YOU WILL HAVE TO CREATE YOUR OWN FOLDER AND THUS CHANGE THE PATH NAME

In [5]:
DIFEV_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits/'

In [6]:
BASIN_HOPPING_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/BASIN_HOPPING_FITS/'

In [7]:
SHGO_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/SHGO_FITS/'

In [8]:
EXHUASTIVE_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Exhuastive fits/'

In [9]:
BOBBY_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/BOBBY FITS/'

In [10]:
POLISH0_PATH='/'

In [11]:
import pickle
from datetime import date
def save_fit(fit, participant,algo):
  tdy=date.today()
  sp=""
  if algo=="direct":
    sp=DIRECT_SAVE_PATH
  elif algo=="diffev":
    sp=DIFEV_SAVE_PATH
  elif algo=="basin_hopping":
    sp=BASIN_HOPPING_SAVE_PATH
  elif algo=="shgo":
    sp=SHGO_SAVE_PATH
  elif algo=="brute":
    sp=EXHUASTIVE_SAVE_PATH
  elif algo=='bobby':
    sp=BOBBY_PATH
  elif algo=='neighbors':
    sp=NEIGHBORS_PATH
  elif algo=='polished_direct':
    sp=POLISHED_DIRECT_PATH



  fname=sp+"participant " + str(participant)+" "+tdy.strftime("%B %d, %Y")+".pickle"
  pickle_out=open(fname,"wb")
  pickle.dump(fit,pickle_out)
  pickle_out.close()



In [12]:
def load_fit(path):
  pickle_in=open(path,"rb") # put the path here
  fit=pickle.load(pickle_in)
  return fit


In [13]:
p = {
    1: 0.03,
    2: 0.06,
    3: 0.09,
    4: 0.12,
    5 : 0.14,
    6: 0.11,
    7: 0.09,
    8: 0.08,
    9: 0.07,
    10: 0.06,
    11: 0.05,
    12: 0.04,
    13: 0.03,
    14: 0.02,
    15: 0.01
}
I=15

In [14]:




def EUTv1(day, i_d, n, stored ):
    s=0
    for j in p.keys:
      s+=(j*(p[j]*10))

    util = n*i_d + (stored-n)*s
    return util




In [15]:


def cutoff_EUT(x):
    if x >= 32:
        return 14
    elif x in range(17, 32):
        return 13
    elif x in range(10, 17):
        return 12
    elif x in range(7, 10):
        return 11
    elif x in range(5, 7):
        return 10
    elif x == 4:
        return 9
    elif x == 3:
        return 8
    elif x == 2:
        return 7
    elif x == 1:
        return 1




In [16]:

def h_prob(day_f, actual_day): # This returns the probability of a sale on the day (d=h) where day is the index variable used in the util represented by f and actual day is the actual day used in the util represented by "day"

  h_rem= 1
  for h in range(actual_day-1, day_f+1, -1):
    h_rem*=sum(p[k] for k in range(1,cutoff_EUT(h)-1))

  h_price=sum(p[j]*j for j in range(cutoff_EUT(day_f),I))
  return (h_price*h_rem)

In [17]:
def EUTv2(day, i_d, stored, n):



  prob_j = sum(h_prob(f,day) for f in range(day-2, 1,-1)) # edit: report just has a big formattinng issue, all is good
  s=0
  for j in range(cutoff_t(day-1), I):
      s+=j*(p[j])


  util= n*i_d + (stored-n)*(s+ prob_j )

  return util




The independent variables in this experiment are i_d, p and n


In [18]:
all_data= pd.read_csv('data/All Data EUT 230519.csv')

In [19]:

SellData= all_data.copy()

In [20]:
columns=list(SellData.columns)
columns

['Subject',
 'Day',
 'Stored68',
 'Price68',
 'Sold68',
 'Predicted68',
 'Stored67',
 'Price67',
 'Sold67',
 'Predicted67',
 'Stored66',
 'Price66',
 'Sold66',
 'Predicted66',
 'Stored65',
 'Price65',
 'Sold65',
 'Predicted65',
 'Stored64',
 'Price64',
 'Sold64',
 'Predicted64',
 'Stored63',
 'Price63',
 'Sold63',
 'Predicted63',
 'Stored62',
 'Price62',
 'Sold62',
 'Predicted62',
 'Stored61',
 'Price61',
 'Sold61',
 'Predicted61',
 'Stored60',
 'Price60',
 'Sold60',
 'Predicted60',
 'Stored59',
 'Price59',
 'Sold59',
 'Predicted59',
 'Stored58',
 'Price58',
 'Sold58',
 'Predicted58',
 'Stored57',
 'Price57',
 'Sold57',
 'Predicted57',
 'Stored56',
 'Price56',
 'Sold56',
 'Predicted56',
 'Stored55',
 'Price55',
 'Sold55',
 'Predicted55',
 'Stored54',
 'Price54',
 'Sold54',
 'Predicted54',
 'Stored53',
 'Price53',
 'Sold53',
 'Predicted53',
 'Stored52',
 'Price52',
 'Sold52',
 'Predicted52',
 'Stored51',
 'Price51',
 'Sold51',
 'Predicted51',
 'Stored50',
 'Price50',
 'Sold50',
 'Predic

In [21]:
for string in columns:
  if "Predicted"  in string:
      columns.remove(string)
print(columns)

['Subject', 'Day', 'Stored68', 'Price68', 'Sold68', 'Stored67', 'Price67', 'Sold67', 'Stored66', 'Price66', 'Sold66', 'Stored65', 'Price65', 'Sold65', 'Stored64', 'Price64', 'Sold64', 'Stored63', 'Price63', 'Sold63', 'Stored62', 'Price62', 'Sold62', 'Stored61', 'Price61', 'Sold61', 'Stored60', 'Price60', 'Sold60', 'Stored59', 'Price59', 'Sold59', 'Stored58', 'Price58', 'Sold58', 'Stored57', 'Price57', 'Sold57', 'Stored56', 'Price56', 'Sold56', 'Stored55', 'Price55', 'Sold55', 'Stored54', 'Price54', 'Sold54', 'Stored53', 'Price53', 'Sold53', 'Stored52', 'Price52', 'Sold52', 'Stored51', 'Price51', 'Sold51', 'Stored50', 'Price50', 'Sold50', 'Stored49', 'Price49', 'Sold49', 'Stored48', 'Price48', 'Sold48', 'Stored47', 'Price47', 'Sold47', 'Stored46', 'Price46', 'Sold46', 'Stored45', 'Price45', 'Sold45', 'Stored44', 'Price44', 'Sold44', 'Stored43', 'Price43', 'Sold43', 'Stored42', 'Price42', 'Sold42', 'Stored41', 'Price41', 'Sold41', 'Stored40', 'Price40', 'Sold40', 'Stored39', 'Price39', '

In [22]:
error_cols=columns[-10:]

In [23]:
columns.remove( 'OEHold')
columns.remove( 'OHold')

columns.remove('OESold')
columns.remove('OSold')
columns.remove( 'MSold')

columns.remove( 'Diff')
columns.remove( 'Tot')
columns.remove( 'MDev')
columns.remove( 'MDev0')
columns.remove('Tot0')



In [24]:
SellData=all_data[columns]
SellData
error_data=all_data[error_cols]

In [25]:
df_days=[]
df_start=SellData.iloc[:,[0]]
splitting_df= SellData.drop("Subject", axis=1)


In [26]:
def split(df):
  cols=list(df.columns)
  stored_cols=[x for x in cols if ("Stored" in x)]
 #for i in DAYS:
  #cols_for_day=[]

  for name in stored_cols:
    print("Stored+ day: ", name)

    stored_i=cols.index(name)
    print("stored index",stored_i)

    new_df= df.iloc[:, list(range(stored_i,stored_i+3))]
    print("length of new df : ", len(new_df.columns))
    print("columns of new df : ", list(new_df.columns))
    new_df= pd.concat([df_start,new_df], axis=1)
    df_days.insert(stored_i, new_df)
  print("length of df_days: ", len(df_days))




In [27]:
split(splitting_df)

Stored+ day:  Stored68
stored index 1
length of new df :  3
columns of new df :  ['Stored68', 'Price68', 'Sold68']
Stored+ day:  Stored67
stored index 4
length of new df :  3
columns of new df :  ['Stored67', 'Price67', 'Sold67']
Stored+ day:  Stored66
stored index 7
length of new df :  3
columns of new df :  ['Stored66', 'Price66', 'Sold66']
Stored+ day:  Stored65
stored index 10
length of new df :  3
columns of new df :  ['Stored65', 'Price65', 'Sold65']
Stored+ day:  Stored64
stored index 13
length of new df :  3
columns of new df :  ['Stored64', 'Price64', 'Sold64']
Stored+ day:  Stored63
stored index 16
length of new df :  3
columns of new df :  ['Stored63', 'Price63', 'Sold63']
Stored+ day:  Stored62
stored index 19
length of new df :  3
columns of new df :  ['Stored62', 'Price62', 'Sold62']
Stored+ day:  Stored61
stored index 22
length of new df :  3
columns of new df :  ['Stored61', 'Price61', 'Sold61']
Stored+ day:  Stored60
stored index 25
length of new df :  3
columns of new

In [28]:
df_days.reverse()

Not really. For EUT, all you are doing is calculating the value of the equation each day when a unit is sold versus when a unit is not sold. If the value of the equation is higher when a unit is not old. If the value of the equation is higher when a unit is sold then the predicted number of units sold is equal to the number of units stored. If the value of the equation is higher when a unit is not sold then the predicted number of units sold for that day is 0.


In [29]:
DAYS=list(range(0,68))

Alright now we are going to be predicting with the PT model. This has parameters and multilpe equation variations. It's prospect theory so a simple gains vs losses.

The prospective value of a sale on day d is determined by the probability that a sale would be made at a larger or smaller price on a subsequent day. If a sale is not made for id, the likelihood of a subsequent sale on day d+v at a smaller value implies that a sale on day d for id would be a gain and the likelihood of a subsequent sale on day d+v at a greater value implies that a sale on day d for id would be a loss


In [30]:
import math

def prelec(p, gamma):
    x=math.exp(-(-math.log(p)) ** gamma)

    return x

In [31]:
prelec(p[4], 1.75)

0.02410153064345989

In [32]:
def PTv1(price, sold, stored, alpha, beta, lam, gam) : #i_d, n, N
  gain = sum((prelec(p[j],gam) * (sold*(price-j))**alpha) for j in range(1, price))
  loss= sum((prelec(p[j],gam) * lam*(sold*(price-j))**beta ) for j in range(price+1, I+1))
  return gain-loss


In [33]:
PTv1(6,1,1,1,1,1.755,1)

3.93575

In [34]:
def h_probPT(day_f, actual_day,gamma, cutoff): # This returns the probability of a sale on the day (d=h) where day is the index variable used in the util represented by f and actual day is the actual day used in the util represented by "day"

  h_rem= 1
  for h in range(actual_day-1, day_f+1, -1):
    h_rem*=sum(prelec(p[k],gamma) for k in range(1,cutoff[h]-1))

  #print("h_Rem for day", actual_day, "is ",h_rem )
  return h_rem

In [35]:
def PTv2(day, price, sold, alpha, beta, lam, gam, c_o ):
  # this is our code and it is copied straight from what's shown on the report. PTv3 is a "working" version however it doesn't adhere directly to the model in the report
  gain=sum(  (prelec(  p[j]+ sum(h_probPTv2(day-1,f,c_o,gam) for f in range(day-2,1,-1)) * p[j], gam ) * (sold*(price-j))**alpha )   for j in range(c_o[day-1],price))

 # print("gain for day", day,"is ", gain)
  loss=sum((prelec(  p[j]+ sum(h_probPTv2(day-1,f,c_o,gam) for f in range(day-2,1,-1)) * p[j] , gam) * lam*(sold*(j-price))**beta )   for j in range(max(c_o[day-1],price+1),I+1))
  #print("loss for day", day,"is ", loss)
  util=gain-loss
 # print("util for day", day, "is", util)
  return util

In [36]:

def h_probPTv2(day, f, c_o,gam):
  prob=1.0
  for h in range(1,f+1):
    prob*=sum(prelec(p[j],gam) for j in range(1,c_o[day-h]))
  return prob

In [37]:
def obj_gain(day, price, sold, alpha, beta, lam, gam, c_o,j):
  gain=sold*(price-j)
  gain=gain**alpha
  gain*=prelec(p[j],gam)
  return gain


In [38]:
def obj_loss(day, price, sold, alpha, beta, lam, gam, c_o,j):
  loss=sold*(j-price)
  loss=loss**beta
  loss*=lam*prelec(p[j],gam)
  return loss

In [39]:

def PTv3(day,price,sold,alpha,beta,lam,gam,c_o):
  util=0
  for j in range(c_o[day-1],price):
    util+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for j in range(max(price+1,c_o[day-1]), I+1):

    util-=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for k in range(1,day+2):

    h_prob=h_probPTv2(day,k-1,c_o,gam)
    #h_prob=h_probPT(day_f=k, actual_day=day,gamma=gam,cutoff=c_o)

    gain_prob=0
    for j in range(c_o[day-k],price):
      gain_prob+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)

    loss_prob=0
    for j in range(max(price+1,c_o[day-k]), I+1):
      loss_prob+=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)

    util+=h_prob*(gain_prob-loss_prob)




  #print("util for day",day,"is ", util)
  return util





In [40]:
testlist = [1]*68

In [41]:
PTv3(7,8,4,1,1,1.755,1,testlist)

6.046399999999999

to compute cutoff prices we iterate through each price (1-15 )and determine if they will result in a gain or loss. I


In [42]:
def max_units(day,price, stored,a,b,l,g,cut_offs):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PTv3(day,price,units,a,b,l,g,cut_offs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [43]:
DAYS

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67]

In [44]:
def info_return(day, participant):
  curr_df= df_days[day].iloc[participant] # data acquiring is fine.
  n_1=curr_df["Sold"+str(day+1)]
  #print("Sold units on day :",day, " is : ", n_1)

  N_d=curr_df["Stored"+str(day+1)]
  #print("Stored on day",day,"is:", N_d)
  i_d=curr_df["Price"+str(day+1)]
 # print("Price on day",day,"is",i_d)
  return(n_1,N_d,i_d)

In [45]:
def cutoff_pt_list(a,b,l,g):#this function generates a list before we make our predicitons
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0:
      continue
    price=cutoffs[day-1]
    prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)
    counts=0
    while (prosp <= 0 and price < 15):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                prosp = PTv3(day,price,1,a,b,l,g,cutoffs)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
   # print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs





In [46]:
def fitPTv2(participant,a,b,l,g):
  predictions=[]                 #[0]*68
  #cut_offs=c_o
  cut_offs=cutoff_pt_list(a,b,l,g)            #cutoff_PT(a,b,l,g,participant)


  for day in DAYS: # 1,2,3
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]

    ''' c_o=cutoff_PT(a,b,l,g,day,cut_offs,stored)
    print("cut off price for day", day,"is ", c_o)
    cut_offs[day-1]=c_o'''
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units(day,price,stored,a,b,l,g, cut_offs)
      else:
        print(".")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  print("DATA ADDED :)")
  return predictions,cut_offs

In [47]:
'''def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  if tw>(68-day):
    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)
    print("util PT : ", util)

  else:
    gain=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(j+1,I+1)))**k for k in range(1,tw-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(1,price))
    loss=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(price+2,I+1)))**k for k in range(1,tw-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(price+1,I+1))
    util=gain-loss
    print("util", util)

  return util
'''





'def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):\n  if tw>(68-day):\n    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)\n    print("util PT : ", util)\n\n  else:\n    gain=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(j+1,I+1)))**k for k in range(1,tw-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(1,price))\n    loss=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(price+2,I+1)))**k for k in range(1,tw-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(price+1,I+1))\n    util=gain-loss\n    print("util", util)\n\n  return util\n'

In [48]:
def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  t=int(tw)

  if t>day: # 68,67,66
    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)
    print("util PT : ", util)
  else:
    gain=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(c_o[t-1],price))
    loss=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(max(c_o[t-1],price+1),I+1))
    util=gain-loss
    print("util day:" ,day,util)
  return util

In [49]:
def cutoff_pt_tw_list(a,b,l,g,tw):
  t=tw
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]
    if day<t:
          prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    else:
          prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                if day<t:
                  prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
                else:
                  prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [50]:
#print(load_fit('/content/drive/MyDrive/Glass Lab Data/PT fits/DIRECT fits/participant 17 September 02, 2023.pickle'))

In [51]:
#count_errorPT_TW([0.72222222, 0.83333333, 1.16666667, 0.16666667,68],16)

In [52]:
'''def cutoff_pt_tw_list(a,b,l,g,tw):
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]

    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1

                prosp = PT_TW(day,price,1,a,b,l,g,tw,cutoffs)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs'''

'def cutoff_pt_tw_list(a,b,l,g,tw):\n  cutoffs=[1]\n  for day in DAYS:#0,1,2,3\n    if day==0: # last day\n      continue\n    price=cutoffs[day-1]\n\n    print(prosp)\n    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)\n\n    counts=0\n    while (prosp <= 0 and price < I):\n                # Continually increment price until it is worthwhile to sell\n                price += 1\n                counts+=1\n\n                prosp = PT_TW(day,price,1,a,b,l,g,tw,cutoffs)\n                print("prosp on day", day, "is ", prosp)\n                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)\n\n\n\n    cutoffs.append(price)\n    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)\n  return cutoffs'

In [53]:
def max_units_PTTW(day,price,stored,a,b,l,g,tw,cutoffs ):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PT_TW(day,price,units,a,b,l,g,tw,cutoffs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [54]:
DAYS

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67]

# THE FUNCTION WE ARE TRYING TO OPTIMIZE


In [55]:
def fitPT_TW(participant,a,b,l,g,tw):
  predictions=[]                 #[0]*68
  cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)
  #cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)            #cutoff_PT(a,b,l,g,participant)

  rev_d=reversed(DAYS)
  #rev_d=DAYS
  for day in rev_d: # 67,66,65...
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units_PTTW(day,price,stored,a,b,l,g,tw,cut_offs)
      else:
        print("cutoff value of", cut_offs[day], "bigger than price", price, "HOLD PREDICTED ")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  print("DATA ADDED :)")
  return predictions,cut_offs

In [56]:
PT_p1,p1_cutoffs=fitPT_TW(0, 1,1,1,1,68)

-11.22
prosp on day 1 is  -9.219999999999999
prosp on day 1 is  -7.22
prosp on day 1 is  -5.220000000000001
prosp on day 1 is  -3.2199999999999998
prosp on day 1 is  -1.22
prosp on day 1 is  0.78
counts from the previous value of 1  is  6 cutoff price is now 7 for day  1
-2.1855
prosp on day 2 is  -0.7354999999999998
prosp on day 2 is  0.7145
counts from the previous value of 7  is  2 cutoff price is now 9 for day  2
-0.38956
prosp on day 3 is  0.8904399999999999
counts from the previous value of 9  is  1 cutoff price is now 10 for day  3
0.22464759999999984
counts from the previous value of 10  is  0 cutoff price is now 10 for day  4
-0.24602839600000015
prosp on day 5 is  0.9639716039999999
counts from the previous value of 10  is  1 cutoff price is now 11 for day  5
0.5383758634
counts from the previous value of 11  is  0 cutoff price is now 11 for day  6
0.22761948388999997
counts from the previous value of 11  is  0 cutoff price is now 11 for day  7
-0.036523438693500054
prosp on 

In [57]:
fitPTv2(0,1,1,1,1)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
DATA ADDED :)


([4,
  0,
  0,
  28,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  16,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  56,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  7,
  9,
  10,
  10,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14])

In [58]:
PT_p1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 56,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 16,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 28,
 0,
 0,
 4]

In [59]:
p1_cutoffs

[1,
 7,
 9,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14]

In [60]:
def h_probPT_DD(day, f, c_o,gam):
  prob=1.0
  for h in range(day-1+1,f+2): # day is added 1  here
    prob*=sum(prelec(p[j],gam) for j in range(1,c_o[day-1])) # I don't think this will work, replacement is c_o[day-h] as that is what Yudelson did TALK tO GLASS # Trying c_o[day-1]
  return prob

In [61]:
def PT_DD(day,price,sold,alpha,beta,lam,gam,c_o, k):
  # since day starts from 0, 1, 2,3,4 for the list DAYS but the equation takes d=1 as the last day so all days in this equation are added 1
  lb_1=c_o[day-1] # this stays the same as it is just the last days cutoff value
  ub_1=price
  gain= sum(
      (
          prelec(p[j],gam)*(sold*(price-j)/(1+k*(day-k+1)))**alpha # day is added 1 here

       )
      for j in range(lb_1, ub_1)
      )
  lb_2=max(lb_1,price+1)
  ub_2=I+1
  loss= sum(

            (
              prelec(
                  (p[j] + sum(h_probPT_DD(day,f,c_o,gam) for f in range(day-1,0,-1))*p[j] # day is added 1 here edit. not anymore
               ),gam)


            )
            * lam*(sold*(j-price)/(1+k*(day-k+1)))**beta # day is added 1 here
            for j in range(lb_2,ub_2)

  )

  util=gain-loss
  print(gain)
  print(loss)
  return util


In [62]:
def cutoff_PT_DD_list(a,b,l,g,k):

  cutoffs=[1] #last day (d=1) but we used day=0
  for day in DAYS:#0,1,2,3 but actual days start 1, 2,3,4
    if day==0: # last day
      continue
    price=cutoffs[day-1] # last day, also the same as the lower bound for the gain in PT-DD


    prosp = PT_DD(day,price,1,a,b,l,g,cutoffs,k) # sold is one as long as 1 will sell its fine
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1

                prosp = PT_DD(day,price,1,a,b,l,g,cutoffs,k)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [63]:
x=cutoff_PT_DD_list(.9,.9,1.2,.5,.5)

0
10.750458991820484
-10.750458991820484
0.09289971006622892
9.258338468901307
prosp on day 1 is  -9.165438758835078
0.2862881663959169
7.854867196361726
prosp on day 1 is  -7.568579029965809
0.5884802182540498
6.557339833933505
prosp on day 1 is  -5.968859615679455
1.0068635348564041
5.382027474778935
prosp on day 1 is  -4.3751639399225315
1.5444626242445152
4.342055962454093
prosp on day 1 is  -2.7975933382095777
2.185502673838152
3.426218130415056
prosp on day 1 is  -1.240715456576904
2.9203556505387467
2.626540905643783
prosp on day 1 is  0.29381474489496373
counts from the previous value of 1  is  7 cutoff price is now 8 for day  1
0
2.7201561967905246
-2.7201561967905246
0.09836292638407718
1.9985208902041784
prosp on day 2 is  -1.9001579638201012
0.27791898190603087
1.3967731496002058
prosp on day 2 is  -1.118854167694175
0.5305536038563231
0.9099534205474109
prosp on day 2 is  -0.3793998166910878
0.8496240123635289
0.5328193375572505
prosp on day 2 is  0.3168046748062784
counts

In [64]:
x

[1,
 8,
 12,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15]

In [65]:
info_return(5,5)

(0, 10, 3)

In [66]:
PT_DD(5,11,4,.99,.99,1.5,.3,[1,7,9,10,10,11,12,13],.5)

0.272540696356297
4.529230397437623


-4.256689701081326

In [67]:
print(list(range(15,3,-1)))

[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4]


In [68]:
parameters_df=pd.read_csv('data/pt_predictions_final.xlsx - Parameters.csv')

In [69]:
def PREDICT_YES():
  for dude in range(0,58):
    a=parameters_df._get_value(dude+1,col='alphaNew')
    b=parameters_df._get_value(dude+1,col='betaNew')
    l=parameters_df._get_value(dude+1,col='lambdaNew')
    g=parameters_df._get_value(dude+1,col='gammaNew')

    fitPTv2(dude,a,b,l,g)

In [70]:
soldOnly = SellData
for name in SellData.columns :
  if 'Sold' not in name:
    soldOnly=soldOnly.drop(name, axis=1)

soldOnly = soldOnly.T
soldOnly = soldOnly.iloc[::-1]
soldOnly

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
Sold1,4,3,3,4,4,4,4,4,4,4,...,3,4,3,10,6,3,2,12,2,2
Sold2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
Sold3,0,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,1,1,0,2,1
Sold4,28,26,12,12,12,12,7,12,6,12,...,5,4,3,20,6,2,1,5,3,1
Sold5,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sold64,0,0,0,0,0,0,0,0,0,0,...,0,1,1,4,1,1,0,1,0,2
Sold65,0,5,2,5,5,7,3,7,0,7,...,3,3,2,4,1,7,5,0,3,4
Sold66,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,1
Sold67,0,0,0,0,0,0,0,0,0,0,...,0,1,3,0,1,0,2,1,1,0


In [71]:
list(soldOnly[0].values)

[4,
 0,
 0,
 28,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 56,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [72]:
import random

In [73]:
from scipy.optimize import minimize

In [74]:
from scipy.optimize import Bounds

In [75]:
from sklearn.metrics import d2_absolute_error_score

In [76]:
from sklearn.metrics import d2_absolute_error_score as d2a

In [77]:
from sklearn.metrics import mean_squared_error

In [78]:
def avg_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)
    stored=vals[1]
    sold=vals[0]
    if stored==0:
      d0+=1
      continue
    else:
      pred=preds[day]
      #pred=predictionsOnly._get_value(day,"Participant"+str(participant))
      day_err=(pred-sold)/stored
      total+=day_err
  return total/(68.0-d0)


In [79]:
def get_loss_one(params,participant):#a,b,l,g
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return mse


In [80]:
def d2a_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return 1-error


In [81]:
def fit_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]



  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  actuals=actuals[::-1]

  error=d2a(actuals,preds)

  return 1-error

In [82]:
def fit_errorPTTW_dir(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)


  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)

  error=d2a(actuals,preds)

  return error

# THE MAIN ERROR FUNCTION WE ARE USING FOR OPTIMIZATION


In [83]:
def count_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]

  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[day]
    day_err=abs(sold-pred)
    total+=day_err
  return total

In [84]:
def count_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)

  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS[::-1]:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[67-day]
    day_err=abs(sold-pred)
    total+=day_err
  return total


In [85]:
def display_errors(four_fits, start,stop):
  avg_errors=[]
  d2a_errors=[]
  mse_errors=[]
  count_errors=[]
  iter=list(zip(four_fits, list(range(start,stop))))
  for fit,participant in iter:
    x=avg_error(fit,participant)
    y=d2a_error(fit,participant)
    z=get_loss_one(fit,participant)
    m=count_error(fit,participant)
    avg_errors.append(x)
    d2a_errors.append(y)
    mse_errors.append(z)
    count_errors.append(m)
  return avg_errors,d2a_errors,mse_errors,count_errors

In [86]:
bound_set=set(list(range(1,69)))

In [87]:
losstest=fit_errorPT_TW(np.array([1,1,1,1,68]),0)

-11.22
prosp on day 1 is  -9.219999999999999
prosp on day 1 is  -7.22
prosp on day 1 is  -5.220000000000001
prosp on day 1 is  -3.2199999999999998
prosp on day 1 is  -1.22
prosp on day 1 is  0.78
counts from the previous value of 1  is  6 cutoff price is now 7 for day  1
-2.1855
prosp on day 2 is  -0.7354999999999998
prosp on day 2 is  0.7145
counts from the previous value of 7  is  2 cutoff price is now 9 for day  2
-0.38956
prosp on day 3 is  0.8904399999999999
counts from the previous value of 9  is  1 cutoff price is now 10 for day  3
0.22464759999999984
counts from the previous value of 10  is  0 cutoff price is now 10 for day  4
-0.24602839600000015
prosp on day 5 is  0.9639716039999999
counts from the previous value of 10  is  1 cutoff price is now 11 for day  5
0.5383758634
counts from the previous value of 11  is  0 cutoff price is now 11 for day  6
0.22761948388999997
counts from the previous value of 11  is  0 cutoff price is now 11 for day  7
-0.036523438693500054
prosp on 

In [88]:
losstest

0.18181818181818177

In [89]:
def default_cb(xk):
  print(xk)
  ret=False
  if count_error(xk,14)<=11:
    ret=True
  return ret

In [90]:
#GUESS=np.array([.5,.5,1,.5,30])
GUESS=[.5,.5,1,.5,30]

In [91]:
from scipy.optimize import differential_evolution
from scipy.optimize import LinearConstraint

In [92]:
def bold_text(text):
    bold_start = '\033[1m'
    bold_end = '\033[0m'
    return bold_start + text + bold_end


print('website: ' + bold_text('bobbyhadz.com') + ' abc 123')


website: bobbyhadz.com abc 123


In [93]:
cb_index=0

In [94]:
our_fits_df=pd.DataFrame()

In [95]:
from time import time
import warnings

In [96]:
def dict_constraint(xk):
  return xk[1]-xk[0]

In [97]:
check=lambda params: params if params[0]<=params[1] else "Alpha greater than Beta"

In [98]:

def differential_evolution_participant(participant,save):
    part=participant
    #init_=init_vertices()
   #init_=init_[0]
    #init_=np.array(init_)
    g=np.array([.5,.5,1,.5,60])
    #g=guess
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(7,68)] # changing the bounds might be the reason why it's messing up, but a and b cannot be greater than 1 keep in mind
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    result = differential_evolution(func=fit_errorPT_TW,x0=g,bounds=bounds, args=(part,), strategy="best1bin", maxiter=30000, tol=0.0002, disp=True,updating="immediate",constraints=de_lc,polish=True,integrality=[False,False,False,False,True] ) # can change back to get_loss_one or avg error if needed
    best_fit = result.x
    print(result.message)
    if save:
      save_fit(best_fit,participant+1,"diffev")
    return best_fit


In [99]:
from scipy.optimize import direct
def direct_participant(participant,dum):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    g=np.array([.5,.5,1,.5,68])
    result = direct(func=fit_errorPT_TW, bounds=bounds, args=(part,), maxiter=10000, locally_biased=False)

    best_fit = result.x
    print (result.message)
    if dum:
      save_fit(check(best_fit),participant+1,'direct')
    return best_fit

In [100]:
from scipy.optimize import shgo
def shgo_participant(participant):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    dc=({'type': 'ineq','fun':dict_constraint})
    g=np.array([.5,.5,1,.5,50])
    result = shgo(func=fit_errorPT_TW, bounds=bounds, args=(part,) , options={'maxiter':10000},constraints=dc)

    best_fit = result.x
    print (result.message)

    return best_fit

In [101]:
from scipy.optimize import minimize
def nelder_mead_participant(participant, guess):
  part = participant
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), bounds=bounds, method='Nelder-Mead', options={"fatol":.15})
  return result.x

cob_eq=lambda x: x[1]-x[0]

def cobyla_participant(participant,guess):
  part = participant
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), method='COBYLA', bounds=bounds, constraints={"type":"ineq", "fun":cob_eq}, options={"tol":.13})
  return result.x

In [102]:
from scipy.optimize import basinhopping
def basin_hopping_participant(participant):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    g=np.array([.5,.5,1,.5,50])
    opt_func=lambda params: fit_errorPT_TW(params,participant)
    result = basinhopping(func=opt_func, minimizer_kwargs={"method":"Nelder-Mead"}, x0=g, niter=10000, interval=75,T=.5)

    best_fit = result.x
    print (result.message)

    return best_fit

In [103]:
def global_polish(participant,starter,count,glazer):
  silly=None
  s=''
  if starter=='diffev':
    silly=differential_evolution_participant
    s='neighbors'
  elif starter=='direct':
    silly= direct_participant
    s='polished_direct'


  ret='didnt work'
  start=silly(participant,False)
  p=None
  if count==1:
    if glazer=='NM':
      p=nelder_mead_participant
    elif glazer=='CB':
      p=cobyla_participant
    end=p(participant, start)
    save_fit(end,participant+1,s)
    ret=end


    ret =check(end)
  elif count==2:
    end1=nelder_mead_participant(participant,start)
    end2=cobyla_participant(participant,start)
    dickle={
        "NM": check(end1),
        "CB":(end2)
    }
    save_fit(dickle,participant+1,s)
    ret=dickle
  return ret





In [104]:
 # works somehow
#dum2=shgo_participant(0) # does not work in time
#dum3=basin_hopping_participant(0) # not converging
#dum2=global_polish(2,'CB')

#b22diff=global_polish(21,"diffev",2,"CB")


In [112]:
def glass_error(params,participant):
  preds=fitPT_TW(participant, params[0], params[1], params[2], params[3], params[4])[0]

  d_plus=0
  d_minus=0
  d0=0
  for day in DAYS[::-1]:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[67-day]
    day_err=sold-pred
    if(day_err)>0:
      d_plus+=day_err
    elif(day_err<0):
      d_minus-=day_err

  return d_plus,d_minus


In [106]:
'''def efficent_fit(participant, target, guess=GUESS, maxiter=50000):
        valid_parameter_ranges= {                        # Dict[str, List[float]] = {

            "a": list(np.arange(
                            .001,#low(prev_best_fit[0],  prev_precision, precision), # here he is setting the minimum value to the value of the guess or the floor value which
                             1, #high(prev_best_fit[0], prev_precision, 1),# + EPS,
                            .001
                )),
            "b": list(np.arange(
                            .001,#low(prev_best_fit[1],  prev_precision, precision),
                            1,  #high(prev_best_fit[1], prev_precision, 1),# + EPS,
                            .001
                )),
            "l": list(np.arange(
                            1,#low(prev_best_fit[2],  prev_precision, 1),
                            2,#high(prev_best_fit[2], prev_precision, 2),# + EPS,
                            .001
                )),

            "g": list(np.arange(
                            .001,#low(prev_best_fit[3],  prev_precision, precision),
                             1,#high(prev_best_fit[3], prev_precision, 1),# + EPS,
                            .001
                )),

            "tw": list(np.arange(2, 68, 1))
        }
        ultimate_best_fit=GUESS
        best_error=count_errorPT_TW(ultimate_best_fit,participant)
        i=0
        queue=[guess]
        visited=[]
        while(best_error>target or i<maxiter ):
          focus_param_index=0 # starts as alpha

          f_prime=ultimate_best_fit[focus_param_index]
          best_fit=ultimate_best_fit

          curr_fit=queue.pop()
          if curr_fit in visited:
            continue

          d_plus,d_minus=glass_error(curr_fit, participant)
          curr_d_plus=d_plus
          f_d_prime=f_prime
          while(d_plus-curr_d_plus<1):
            curr_fit[focus_param_index]+=.001   #
            f_d_prime+=.001   #
            curr_d_plus=glass_error(curr_fit,participant)


          #curr_error=count_errorPT_TW(curr_fit,participant)

          curr_d_minus=glass_error(curr_fit,participant)[1]
          if(curr_d_minus<=d_minus):
            best_fit=curr_fit
            f_prime=f_d_prime


          if (curr_d_plus==0 and f_d_prime>f_prime):
                break # break changing alpha

          while True:


            f_d_prime+=.001
            curr_fit[]

            d_plus,_=glass_error(curr_fit,participant)


'''



'def efficent_fit(participant, target, guess=GUESS, maxiter=50000):\n        valid_parameter_ranges= {                        # Dict[str, List[float]] = {\n\n            "a": list(np.arange(\n                            .001,#low(prev_best_fit[0],  prev_precision, precision), # here he is setting the minimum value to the value of the guess or the floor value which\n                             1, #high(prev_best_fit[0], prev_precision, 1),# + EPS,\n                            .001\n                )),\n            "b": list(np.arange(\n                            .001,#low(prev_best_fit[1],  prev_precision, precision),\n                            1,  #high(prev_best_fit[1], prev_precision, 1),# + EPS,\n                            .001\n                )),\n            "l": list(np.arange(\n                            1,#low(prev_best_fit[2],  prev_precision, 1),\n                            2,#high(prev_best_fit[2], prev_precision, 2),# + EPS,\n                            .001\n      

In [117]:
glass_error([.5,.5,1,0,68],0)

-26.925988961697215
prosp on day 1 is  -23.437272422351533
prosp on day 1 is  -19.74393585564041
prosp on day 1 is  -15.92082055662651
prosp on day 1 is  -12.009066643038146
prosp on day 1 is  -8.037185890166825
prosp on day 1 is  -4.027675407677661
prosp on day 1 is  -5.551115123125783e-16
prosp on day 1 is  4.027675407677659
counts from the previous value of 1  is  8 cutoff price is now 9 for day  1
-2.042813402055196
prosp on day 2 is  6.395245218424813
counts from the previous value of 9  is  1 cutoff price is now 10 for day  2
23.99854800728632
counts from the previous value of 10  is  0 cutoff price is now 10 for day  3
83.49960429105842
counts from the previous value of 10  is  0 cutoff price is now 10 for day  4
280.50254230411997
counts from the previous value of 10  is  0 cutoff price is now 10 for day  5
932.7625190125163
counts from the previous value of 10  is  0 cutoff price is now 10 for day  6
3092.339840582362
counts from the previous value of 10  is  0 cutoff price is

(0, 160)

In [119]:
glass_error([.5,.5,1,.9,68],0)

-5.730179275412393
prosp on day 1 is  -4.940340072103293
prosp on day 1 is  -4.007359317401519
prosp on day 1 is  -2.9474533911269876
prosp on day 1 is  -1.7959391774669338
prosp on day 1 is  -0.6415731337616591
prosp on day 1 is  0.45097747042517083
counts from the previous value of 1  is  6 cutoff price is now 7 for day  1
-1.6113655068286459
prosp on day 2 is  -0.6197348800564746
prosp on day 2 is  0.35949908709298195
counts from the previous value of 7  is  2 cutoff price is now 9 for day  2
-0.4564325782923002
prosp on day 3 is  0.5660344721194988
counts from the previous value of 9  is  1 cutoff price is now 10 for day  3
0.11065377575095137
counts from the previous value of 10  is  0 cutoff price is now 10 for day  4
-0.2418226220850268
prosp on day 5 is  1.0399745477316025
counts from the previous value of 10  is  1 cutoff price is now 11 for day  5
0.7947945590474066
counts from the previous value of 11  is  0 cutoff price is now 11 for day  6
0.6215897565999331
counts from th

(0, 56)

In [ ]:
def change(index,d_plus):
  if d_plus:
    if index==0:
      return .001
    else:
      return -.001
  else:
    if (index==0):
      return -.001
    else:
      return .001

In [ ]:
def efficient_fit_ayan_jaiman(participant, target=0, guess=GUESS, maxiter=50000):
  '''
  Finds the best fit using an efficient version of brute force devised by Dr. Glass.

  params:
    participant   :       int             index of the participant
    target        :       int             the optimal count_error value that we want to reach
    guess         :       list<float>     the initial parameter values (usually are the previous participant's best fits)
    maxiter       :       int             maximum number of iterations, to prevent CPU burden


  notes:  what's target for? I thought we improve as much as possible rather than stopping at a set value
          Why do we have a focus_param_index? we are starting with just alpha but we are going to eventually change both alpha and beta at the same time.
          So, best_fit and curr_fit are the *overall* equivalents of f_d_prime and f_prime?
  '''

  # our ultimate best fit starts at the initial param values
  ultimate_best_fit = guess

  # find the count_error we are starting with (this is our f_prime)
  f_prime = count_errorPT_TW(ultimate_best_fit,participant)

  # to track iterations
  i = 0

  # for fitting each parameter sequentially
  curr_fit = [guess].pop()

  # to log our parameter values
  visited = []


  # MAIN OPTIMIZATION LOOP
  while(f_prime>target or i<maxiter ):

    # set the focus parameter (starting with alpha) (see notes)
    focus_param_index = 0

    # record our baseline value for the parameter
    og_baseline_param = ultimate_best_fit[focus_param_index]

    # update f_prime
    f_prime = count_errorPT_TW(ultimate_best_fit,participant)

    # ??
    best_fit=ultimate_best_fit


    # QUESTION is this to skip over parameter values we have seen before? can we discuss this
    if curr_fit in visited:
      continue

    # calculate our D+ and D- values (what must be reduced)
    d_plus,d_minus = glass_error(curr_fit, participant)
    curr_d_plus = d_plus
    curr_d_minus = d_minus

    # ??
    baseline_param = og_baseline_param

    # DECLARE OUR F"
    f_d_prime = f_prime

    # OPTIMIZING D+
    while (f_d_prime - d_plus < f_prime and d_plus > 0): # our D+ condition

      # THIS IS WHILE LOOP #2
      while (d_plus - curr_d_plus < 1): # First, change alpha to reduce D+ by 1

        # increment our baseline alpha value
        baseline_param -= .001  # decrease alpha -> smaller predicted values -> decreased D+

        # update curr_fit with our new alpha value
        curr_fit = np.array( curr_fit[:focus_param_index] + [baseline_param] + curr_fit[focus_param_index:] ) # [baseline_param]+list(curr_fit)[~focus_param_index])

        # calculate our new F", D+, D-
        f_d_prime = count_errorPT_TW(curr_fit,participant)
        curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

        # log our new parameter values
        visited.append(curr_fit)

      # END WHILE LOOP #2

      # Now that D+ has been reduced, D- may or may not have been increased (since predicted values decreased)
      # So if D- hasn't increased, we have a better fit
      if (curr_d_minus <= d_minus):
        best_fit = curr_fit
        if (f_d_prime < f_prime):
          f_prime = f_d_prime # best error after step 1 is the error reached if d_minus doesn't change

      else:
        # if D- has been affected, do NOT update f_prime
        continue


      d_plus=curr_d_plus
      d_minus=curr_d_minus

  # END D+ OPTIMIZATION


  # OPTIMIZING D-
  while (f_d_prime - d_minus < f_prime and d_minus > 0): # our D- condition


    # THIS IS WHILE LOOP #2
    while (d_minus - curr_d_minus < 1): # First, change alpha to reduce D- by 1

      # increment our baseline alpha value
      baseline_param += .001  # increase alpha -> greater predicted values -> decreased D-

      # update curr_fit with our new alpha value
      curr_fit=np.array( curr_fit[:focus_param_index] + [baseline_param] + curr_fit[focus_param_index:] ) # [baseline_param]+list(curr_fit)[~focus_param_index])

      # calculate our new F", D+, D-
      f_d_prime = count_errorPT_TW(curr_fit,participant)
      curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

      # log our new parameter values
      visited.append(curr_fit)

    # END WHILE LOOP #2

    # Now that D- has been reduced, D+ may or may not have been increased (since predicted values decreased)
    # So if D+ hasn't increased, we have a better fit
    if (curr_d_plus <= d_plus):
      best_fit = curr_fit
      # sanity check
      if (f_d_prime < f_prime):
        f_prime = f_d_prime # best error after step 1 is the error reached if D+ doesn't change
    else:
      # if D- has been affected, do NOT update f_prime
      continue


    d_plus = curr_d_plus
    d_minus = curr_d_minus

  # END D- OPTIMIZATION


  # NOW WE ARE LOOKING AT BETA AND ALPHA TOGETHER
  focus_param_index = 1
  baseline_param = curr_fit[focus_param_index]


  # STEP 2 STEP 2 STEP 2 STEP 2
  # idea is to start from the middle of the alpha search space

  # D+ LOOP (step 2)
  done = False
  while (curr_fit[1] < 1): # incrementing beta

    while (curr_fit[0] > 0): # decrementing alpha
      curr_fit[0] -= .001

      # alpha has to be < beta
      if (curr_fit[0] > curr_fit[1]):
        continue

      # calculate our new F", D+, D-
      curr_d_plus,_ = glass_error(curr_fit,participant)

      if (d_plus - curr_d_plus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        # reduce D+ until f" - D+ > f'
        else:
          done = True
          break

        d_plus = curr_d_plus
        # if improved D+ as much as we can without starting to get worse, stop
        if (d_plus == 0): break

    # reduce D+ until f" - D+ > f'
    if done:
      break

    curr_fit[1] += .001

  # D- LOOP (step 2)
  done = False
  while (curr_fit[1] > 0): # decrementing beta

    while (curr_fit[0] < 1): # incrementing alpha
      curr_fit[0] += .001

      # alpha has to be < beta
      if (curr_fit[0] > curr_fit[1]):
        curr_fit[0] -= .001
        done = True
        break

      # calculate our new F", D-
      _,curr_d_minus = glass_error(curr_fit,participant)

      if (d_minus - curr_d_minus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        # reduce D- until f" - D- > f'
        else:
          done = True
          break

        d_minus = curr_d_minus
        # if improved D- as much as we can without starting to get worse, stop
        if (d_minus == 0): break

    # reduce D+ until f" - D+ > f'
    if done:
      break

    curr_fit[1] -= .001


  # STEP 3 STEP 3 STEP 3
  curr_fit, f_prime = step3(curr_fit,f_prime)
  # Just step 2 but for everything

  # while bounds for i=3
    # while bounds for j=2
      # while bounds for j=1 ...
        # increment 1

        # # continue if alpha < beta
        # if (curr_fit[0] < curr_fit[1]):
        #   continue

        # # calculate our new F", D+, D-
        # curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

        # if (d_plus - curr_d_plus > 1):
        #   f_d_prime = count_errorPT_TW(curr_fit,participant)
        #   if (f_d_prime < f_prime):
        #     f_prime = f_d_prime
        #   d_plus = curr_d_plus
    # increment 2
  # increment 3
    
  # if j > 1 then recurse
  # else do the meaty stuff



In [ ]:
def stepParam(index, d):
    '''

    model parameters: alpha, beta, lambda, gamma, (tw)

    index: index of the param
    d: boolean, d+ is True, d- is False

    Returns: float for increment value
    '''
    
    # alpha
    if index == 0:
        return -.001 if d else +.001
    # beta, lambda, gamma (opposite of alpha)
    elif index == 1 or index == 2 or index==3:
        return stepParam(0, not d)



parameter bounds:

alpha (0,1)
beta (0,1)

alpha < beta

lambda (1,2)
gamma (0,1)

tw (1,68)


In [ ]:
def boundParam(index):
    if (index != 2 and index != 4):
        return 0, 1

In [5]:
def recursion(curr_fit, f_prime, participant, index=3):
  '''
  Does step 3 through recursion.

  Parameters:
    curr_fit: list<float> = current parameter values [alpha, beta, lambda, gamma, tw]
    f_prime: float = current best error value
    participant: int = index of the participant to fit
    index: int = index of the parameter to start with (default is gamma)

  Returns:
    curr_fit: list<float> = new best parameter values
    f_prime: float = new best error value


  Logic:
    1. Start with the first parameter (alpha). Until we get best possible fit by changing just alpha:
      1.a. Decrement alpha until D+ is reduced by 1
        1.a.i. Calculate F" and replace f_prime if better fit
    2. Recurse to the next parameter (beta) and until best possible fit:
      2.a Repeat step 1
      2.b. Increment beta
    3. Recurse to the next parameter (lambda) and until best possible fit:
      3.a. Repeat steps 1 and 2
      3.b. Increment lamda
    4. Recurse to the next parameter (gamma) and until best possible fit:
      4.a. Repeat steps 1, 2, and 3
      4.b. Increment gamma
    5. Repeat steps 1-4 but for D- instead of D+ (switch increment to decrement and vice versa)
  '''

  # BASE CASE
  if (index < 0):
    return curr_fit, f_prime
  
  # D+ LOOP
  done = False
  while (curr_fit[index] < boundParam(index)[1] and curr_fit[index] > boundParam(index)[0] and (done==False if index==3 else True)):
    if (index == 0): # if alpha
      # increment alpha
      curr_fit[index] += stepParam(index, True)

      # calculate our new F", D+, D-
      curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

      # conditions to stop
      if (d_plus - curr_d_plus > 1 and curr_d_minus == d_minus): # refer to step 1, end of first paragraph
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        else:
          # stop D+ optimization if we are starting to get a worse fit
          done = True

        d_plus = curr_d_plus
        # if improved D+ as much as we can without starting to get worse, stop
        if (d_plus == 0): done=True
    else: # not alpha
      curr_fit, f_prime = recursion(curr_fit, f_prime, participant, index-1)
    
    # increment the parameter
    curr_fit[index] += stepParam(index, True)

  # D- LOOP
  done = False
  while (curr_fit[index] < boundParam(index)[1] and curr_fit[index] > boundParam(index)[0] and (done==False if index==3 else True)):
    if (index == 0): # if alpha
      # increment alpha
      curr_fit[index] += stepParam(index, True)

      # calculate our new F", D+, D-
      curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

      # conditions to stop
      if (d_minus - curr_d_minus > 1 and curr_d_plus == d_plus): # refer to step 1, end of first paragraph
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        else:
          # stop D- optimization if we are starting to get a worse fit
          done = True

        d_minus = curr_d_plus
        # if improved D- as much as we can without starting to get worse, stop
        if (d_minus == 0): done=True
    else: # not alpha
      curr_fit, f_prime = recursion(curr_fit, f_prime, participant, index-1)

    # increment the parameter
    curr_fit[index] += stepParam(index, False)

  return curr_fit, f_prime
  '''
  # BASE CASE - we have gone through all parameters (including alpha)
  if (index < 0):
    return curr_fit, f_prime

  done = False
  while (curr_fit[index] < boundParam(index)[1] and curr_fit[index] > boundParam(index)[0]):
    if (index == 0): # if alpha
      curr_fit[index] += stepParam(index, True)

      # calculate our new F", D+, D-
      curr_d_plus,_ = glass_error(curr_fit,participant)

      # conditions to stop
      if (d_plus - curr_d_plus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        # reduce D+ until f" - D+ > f'
        else:
          # stop if we are starting to get a worse fit
          done = True
          break

        d_plus = curr_d_plus
        # if improved D+ as much as we can without starting to get worse, stop
        if (d_plus == 0): break
    else:
      curr_fit, f_prime = recursion(index-1, curr_fit, f_prime, participant)
      
    curr_fit[index] += stepParam(index, True)

  return curr_fit, f_prime
  '''


In [4]:
def step4():
    for i in range(1, 57):
        recursion()

In [3]:
x = 0
y=10
while (x < 10 and (x<5 if y==10)):
  print(x)
  x += 1

0
1
2
3
4


In [ ]:
'''
done = False
done_prime new list[i-1]
i=4
    while done = False
        if j=3 within bounds:
            STEP curr_fit[3]
            calculate curr_d
            if curr_d changed by 1:
                calculate f_d_prime
                if f_d_prime < f_prime:
                    f_prime = f_d_prime
                else:
                    done = True
                    break
            d = curr_d
            if (d == 0):
                done = True
                break
        if j=2 within bounds:
            STEP curr_fit[2]
            calculate curr_d
            if curr_d changed by 1:
                calculate f_d_prime
                if f_d_prime < f_prime:
                    f_prime = f_d_prime
                else:
                    done = True
                    break
            d = curr_d
            if (d == 0):
                done = True
                break
        if j=1 within bounds:
            STEP curr_fit[1]

            calculate curr_d
            if curr_d changed by 1:
            
                calculate f_d_prime
                if f_d_prime < f_prime:
                    f_prime = f_d_prime
                else:
                    done = True
                    break
                    
            d = curr_d
            if (d == 0):
                done = True
                break
        
        if (all done_prime are True):
            done = True
            break
    STEP curr_fit[4]

    
'''

# recursion probly best


In [ ]:
'''
X
step i
    
    19
    step j3 within boundParam(3)
    check conditions
    - d changed by 1
    - f_d_prime better
        - else stop
    - d = 0
    
    14
    step j2 within boundParam(2)
    check conditions
    - d changed by 1
    - f_d_prime better
        - else stop
    - d = 0
    
    9
    step j1 within boundParam(1)
    check conditions
    - d changed by 1
    - f_d_prime better
        - else stop
    - d = 0
'''

In [ ]:
def step2(i, j, curr_fit, f_prime, participant):
  '''
  helper function for efficient_fit_ayan_jaiman
  '''
  # D+ LOOP (step 2)
  done = False
  while (curr_fit[i] > boundParam(i)[0] and curr_fit[i] < boundParam(i)[1]):

    while (curr_fit[j] > boundParam(j)[0] and curr_fit[j] < boundParam(j)[1]): # decrementing alpha
      curr_fit[j] += stepParam(j, True)

      # alpha has to be < beta
      if (curr_fit[0] > curr_fit[1]):
        continue

      # calculate our new F", D+, D-
      curr_d_plus,_ = glass_error(curr_fit,participant)

      if (d_plus - curr_d_plus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        # reduce D+ until f" - D+ > f'
        else:
          done = True
          break

        d_plus = curr_d_plus
        # if improved D+ as much as we can without starting to get worse, stop
        if (d_plus == 0): break

    # reduce D+ until f" - D+ > f'
    if done:
      break

    curr_fit[i] += stepParam(j, True)

  # D- LOOP (step 2)
  done = False
  while (curr_fit[i] > boundParam(i)[0] and curr_fit[i] < boundParam(i)[1]): # decrementing beta

    while (curr_fit[j] > boundParam(j)[0] and curr_fit[j] < boundParam(j)[1]): # incrementing alpha
      curr_fit[j] += stepParam(j, False)

      # alpha has to be < beta
      if (curr_fit[0] > curr_fit[1]):
        if (j==0): curr_fit[j] -= .001
        done = True
        break

      # calculate our new F", D-
      _,curr_d_minus = glass_error(curr_fit,participant)

      if (d_minus - curr_d_minus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)

        if (f_d_prime < f_prime):
          f_prime = f_d_prime
        # reduce D- until f" - D- > f'
        else:
          done = True
          break

        d_minus = curr_d_minus
        # if improved D- as much as we can without starting to get worse, stop
        if (d_minus == 0): break

    # reduce D+ until f" - D+ > f'
    if done:
      break

    curr_fit[i] += stepParam(i, False)

  return curr_fit, f_prime

In [ ]:
def step3(curr_fit, f_prime, participant):
    for i in range(1, len(curr_fit)):
        for j in range(0, i):
            curr_fit, f_prime = step2(i,j, curr_fit, f_prime, participant)

    return curr_fit, f_prime

Glass questions


In [ ]:
!pip install Py-BOBYQA

In [ ]:
import pybobyqa as bob

In [ ]:
def bobyqa_participant(participant):

  g=np.array([.5,.5,1,.5,60])
  #bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]
  b=(np.array([0,0,1,0,2]),np.array([1,1,2,1,68]))
  error_fn=fit_errorPT_TW
  result=bob.solve(error_fn,g,args=(participant,) ,bounds=b,seek_global_minimum=True,print_progress=True,maxfun=5000, rhobeg=.4)
  answer=0

  #print("Success!")
  print(result.msg, result.nf)
  print(result)
  answer=result.x
  save_fit(answer,participant+1,'bobby')

  return answer


In [ ]:
!pip install nlopt

In [ ]:
import nlopt
def nlopt_participant(participant):
  #algorithm=nlopt.NLOPT_GN_CRS2_LM # NLOPT_GN_CRS2_LM CRS technique...
  algorithm="NLOPT_GN_CRS2_LM"
  opt = nlopt.opt(algorithm, 5)
  func=lambda params: fit_errorPT_TW(params,participant)
  opt.set_min_objective(func)
  lower_bounds=[0,0,1,0,2]
  upper_bounds=[1,1,2,1,68]
  opt.set_lower_bounds(lower_bounds)
  opt.set_upper_bounds(upper_bounds)
  stopval=0.15
  opt.set_stopval(stopval)
  maxtime=25200
  # figure out constraints
  opt.set_maxtime(maxtime)

  opt.optimize(GUESS)
  opt_val = opt.last_optimum_value()
  result = opt.last_optimize_result()
  return result


In [ ]:
!pip install dlib

In [ ]:
import dlib

In [ ]:
def dlib_participant(participant):
  func=lambda a,b,l,g,tw: fit_errorPT_TW(np.array([a,b,l,g,tw]),participant)
  lower_bounds=[0,0,1,0,2]
  upper_bounds=[1,1,2,1,68]
  x,y = dlib.find_min_global(func,
                           lower_bounds,  # Lower bound constraints on x0 and x1 respectively
                           upper_bounds,    # Upper bound constraints on x0 and x1 respectively
                           50000)         # The number of times find_min_global() will call func
  return x


In [ ]:
!pip install nevergrad

In [ ]:
import nevergrad as ng

In [ ]:
def modified_func(a,b,l,g,tw,participant):
  return fit_errorPT_TW(np.array([a,b,l,g,tw]),participant)

In [ ]:
# prompt: Define a python function that will return the optimized result of fit_error_PTTW for all 5 parameters using nevergrad

def nevergrad_participant(participant):
  fit_func=lambda a,b,l,g,tw: modified_func(a,b,l,g,tw,participant)

  arg_a=ng.p.Scalar().set_bounds(0,1)
  arg_b=ng.p.Scalar().set_bounds(0,1)
  arg_l=ng.p.Scalar().set_bounds(0,2)
  arg_g=ng.p.Scalar().set_bounds(0,1)
  arg_tw=ng.p.Scalar().set_bounds(0,68).set_integer_casting()

  params=ng.p.Instrumentation(arg_a,arg_b,arg_l, arg_g,arg_tw)

  optimizer = ng.optimizers.NGOpt(params,budget=30000)

  result = optimizer.minimize(fit_func)
  save_fit(result.value,participant+1,"brute")
  return result.value


In [ ]:
#count_errorPT_TW(bf23,22)

In [ ]:
#bf23=nlopt_participant(22)

In [ ]:
#bf23=bobyqa_participant(22)

In [ ]:
#bf23

In [ ]:
#count_errorPT_TW(bf23,22)

In [ ]:
#bfd23=direct_participant(22)

In [ ]:
PARAM_LIST=["a","b","l","g","tw"]

In [ ]:
import itertools
from itertools import product

In [ ]:
import tqdm

In [ ]:
MAX_ITER=7000

In [ ]:
def exhaustive_fit_with_guess_one_subject(participant,precision,guess,target):
                                #subject:        int,

        """Performs the exhaustive fit algorithm for one subject and saves the best fit.
        The exhaustive fit algorithm consists of iterating through all possible parameter values (with a given level of precision) and accepting the best.
        Inputs:
            subject: the participant's number in the dataframe.
            precision: the amount to increment each value when iterating through all possible values.
            verbose: set to True to get progress bars for the fitting.
            error_type: should the error be calculated as the absolute difference
                        between the prediction and the amount, or as
                        the difference in proportion of goods sold. report.docx
                        seems to use proportional.
            tw: if provided, tw will stay at this constant level."""

        #prev_best_fit: Parameters = self.best_fits[subject]
        #initial_pred = self.predict_one_subject(subject=subject, fit=prev_best_fit)
        #lowest_error = self.mean_error_one_subject_proportion(subject, initial_pred)
        prev_best_fit=guess #array/list
        best_fit = prev_best_fit #array/ list
        initial_pred=fitPT_TW(participant,guess[0],guess[1],guess[2],guess[3],guess[4])[0] #a,b,l,g,tw

        # Set lowest error to initial error

       #lowest_error=fit_errorPT_TW(prev_best_fit,participant)  # d2a error
        lowest_error=count_errorPT_TW(prev_best_fit,participant)
        # Get lists of all possible values for all free params
        low  = lambda fit, old_precision, floor: max(floor, fit - old_precision/2) if fit else 0  #basically setting the floor which is 0.01,
        high = lambda fit, old_precision, ceil:  min(ceil,  fit + old_precision/2) if fit else 0  # setting the ceil which is 1 for except lam which it is 2
        valid_parameter_ranges= {                        # Dict[str, List[float]] = {

            "a": list(np.arange(
                            .01,#low(prev_best_fit[0],  prev_precision, precision), # here he is setting the minimum value to the value of the guess or the floor value which
                             1, #high(prev_best_fit[0], prev_precision, 1),# + EPS,
                            precision
                )),
            "b": list(np.arange(
                            .01,#low(prev_best_fit[1],  prev_precision, precision),
                            1,  #high(prev_best_fit[1], prev_precision, 1),# + EPS,
                            precision
                )),
            "l": list(np.arange(
                            1,#low(prev_best_fit[2],  prev_precision, 1),
                            2,#high(prev_best_fit[2], prev_precision, 2),# + EPS,
                            precision
                )),

            "g": list(np.arange(
                            .01,#low(prev_best_fit[3],  prev_precision, precision),
                             1,#high(prev_best_fit[3], prev_precision, 1),# + EPS,
                            precision
                )),

            "tw": list(np.arange(2, 68, 1))
        }
        #if tw is not None:
         # valid_parameter_ranges["tw"] = [tw]

        # Remove data on non-free params:
      #  ranges: List[List[Any]] = [valid_parameter_ranges[param] if param in self.free_params
         #                                                        else [None]
         #                                                        for param in PARAM_LIST ]
        ranges=[valid_parameter_ranges[param] for param in PARAM_LIST]
        # Get all possible values via cartesian product
        all_possible_fits = list(product(*ranges)) # takes every possible range

        # Iterate through every possible value
        iterations = 1
        for range in ranges:
            iterations *= len(range)
       # for fit in tqdm(all_possible_fits,

                      #  leave=False,
                       # total=iterations,
                       # desc="Attempting all fits..."):
        count=0
        for fit in all_possible_fits:
            count+=1
            print("iter:",count)
            if (lowest_error<=target) or (count>MAX_ITER):
              break
            # Skip fits where a > b
            if fit[0] > fit[1]:
                continue
            # Keep fits where xl-.1 <= xg <= xl
         #   if (fit[2] is not None and fit[3] is not None) and (fit[2] > fit[3] or fit[2] < fit[3]-0.1):
               # continue

            # Predict sale amounts based on fit

           # predictions: List[int] = self.predict_one_subject(subject, fit_params)
            fit_params=fit

            preds=fitPT_TW(participant,fit_params[0],fit_params[1],fit_params[2],fit_params[3],fit_params[4])
            # Get error for the given fit
            #error: float = error_fn(subject, predictions)
            #error=fit_errorPT_TW(np.array(fit_params),participant)
            error=count_errorPT_TW(np.array(fit_params),participant)
            # Check if it's the best so far:
            if error < lowest_error:
                lowest_error = error
                best_fit  = fit_params

            elif error == lowest_error:
                if fit_params[0] is not None and fit_params[0] >= best_fit[0]:
                    best_fit = fit_params
                elif not fit_params[0]:
                    best_fit = fit_params
                #self.all_best_fits[subject].append(fit_params)

        save_fit(best_fit,participant+1,"brute")
        return best_fit



In [ ]:
ranges=[(0,1), (0,1), (1,2), (0,1), (2,68)]

In [ ]:

def get_neighbors(params,precision):
  params=list(params)
  neighbuurs=[]


  for index,p in enumerate(params):
    add=precision
    if index==4:
      add=1
    bound_up=lambda p,i: p if p+add>=ranges[i][1] else p+add
    bound_down=lambda p,i: p if p-add<=ranges[i][0] else p-add


    poop=params.copy()
    up_n=poop.copy()
    up_n[index]=bound_up(p,index)
    down_n=poop.copy()
    down_n[index]=bound_down(p,index)
    neighbuurs.append(up_n)
    neighbuurs.append(down_n)
# ret=[np.array(l) for l in neighbuurs]

  return ret




In [ ]:
def bfs_fit_one_subject(participant, guess, target, precision



):


        # Ensure greedy fit has a starting point

        # Set correct error function
        #error_fn = self.get_error_fn(error_type)
        error_fn=count_errorPT_TW

                # Get initial error

        start_error = error_fn(guess,participant)
        best_fit=guess
        best_error=start_error

        # Keep track of every visited node, along with its error
        #visited: Dict[Parameters, bool] = {}
        visited={} #(np.array,error) : bool

        # Keep track of the current best candidate
        lowest_error = start_error

        # BFS keeps track of a queue of nodes to be visited, and checks the oldest first
        #bfs_queue: List[Tuple[Parameters, float]] = [(start_fit, float('inf'))]
        bfs_queue=[(guess,float('inf'))]
        iter=0

        #while len(bfs_queue) != 0:
        while iter<1000 and lowest_error>target:
            # Get current node
            #print(f'{len(bfs_queue)} nodes remaining / {len(visited)} visited', end="\r")
            current_fit, parent_error = bfs_queue.pop(0)

            if visited.get((tuple(current_fit),parent_error)): #If you've already been here...
                continue

              # marking your spot

            #predictions = self.predict_one_subject(subject, current)
            error = error_fn(current_fit, participant)
            visited[(tuple(current_fit),error)] = True

            if error > parent_error:
                continue

            if error < lowest_error:
                lowest_error = error
                best_fit = current_fit
               # self.all_best_fits[subject] = [current]
            elif error == lowest_error:
                best_fit=current_fit


            all_neighbors = get_neighbors(current_fit, precision)

            # Iterate through each neighbor
            best_n=100
            for neighbor in all_neighbors:

                # Get fit as parameter
                #neighbor_fit = Parameters(*neighbor)
                # If fit uses a and b, a must be less than b
                if neighbor[0] is not None and neighbor[1] is not None and neighbor[0] > neighbor[1]:
                    continue
                # Skip visited nodes
                if visited.get(tuple(neighbor)):
                    continue
                n_error=error_fn(neighbor, participant)
                visited[(tuple(neighbor),n_error)]=True
                if n_error<best_n:
                  bfs_queue.append((neighbor, error))
                # Get predictions and errors for each neighbor
        save_fit(best_fit, participant, "neighbors")
        return best_fit



In [ ]:
#bf23=exhaustive_fit_with_guess_one_subject(22,.1,np.array([.5,.5,1.5,.5,50]),28)
#print("official fit:     "+bf23)


In [ ]:
#bf23

In [ ]:
#bf23=direct_participant(22)

In [ ]:
#bf23=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Exhuastive fits/participant 23 December 10, 2023.pickle')

In [ ]:
def takeSecond(elem):
  return elem[1]

In [ ]:
def sorted_ranking(lst):
  entire_tuple_list = []
  for i in range (len(lst)):
    tempTup = (i,count_error(lst[i],i))
    entire_tuple_list.append(tempTup)
  entire_tuple_list.sort(key=takeSecond)
  return entire_tuple_list

In [ ]:
import os

In [ ]:
fit_data=os.listdir('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits')

In [ ]:
fit_data[1][12:14]

In [ ]:
fit_data

In [ ]:
diffev_fits=[(int(path[12:14]),load_fit(DIFEV_SAVE_PATH+path)) for path in fit_data]

In [ ]:
df_days

In [ ]:
diffev_fits

In [ ]:
def sort_tuples(lst):
    return sorted(lst, key=lambda x: x[0])


In [ ]:
pt_tw_errors=[(i-1,count_errorPT_TW(fit,i-1)) for (i,fit) in diffev_fits]

In [ ]:
pt_tw_errors

In [ ]:
sorted_errors=sort_tuples(pt_tw_errors)

In [ ]:
len(sorted_errors
    )

In [ ]:
sorted_errors[33]

In [ ]:
sorted_errors

In [ ]:
sorted_errors.remove(sorted_errors[24])
sorted_errors.remove(sorted_errors[33])
sorted_errors.remove(sorted_errors[6])


In [ ]:
our_errors=pd.read_csv('/content/drive/MyDrive/Glass Lab Data/our_errors - fixed our_errors.csv')

In [ ]:
our_errors

In [ ]:
our_errors=our_errors.drop(["Unnamed: 0"], axis=1)

In [ ]:
our_errors

In [ ]:
z=zip(list(our_errors.index),list(our_errors['our_error'].values))

In [ ]:
PT_errs=list(z)

In [ ]:
PT_errs

In [ ]:
sorted_errors

In [ ]:
PT_better=[]
PT_TW_better=[]
equals=[]
for i in range(57):
  Pt_tw=sorted_errors[i]
  Pt=PT_errs[i]
  if Pt_tw[1]<Pt[1]:
    PT_TW_better.append(i)
  elif Pt_tw[1]==Pt[1]:
    equals.append(i)
  else:
    PT_better.append(i)


In [ ]:
PT_better

In [ ]:
bf22=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/participant 22 December 27, 2023.pickle')

In [ ]:
count_errorPT_TW(bf22['CB'],21)

ALL OF THE ABOVE MUST BE CHANGED

improved pt_tw

participant 2 (i=1) fit=
array([ 0.40763567, 0.43357354, 1.03015975, 0.96945102, 14.00001488])
count error=1


In [ ]:
loaded_bf23_dif= load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/participant 23 January 02, 2024.pickle')
loaded_bf23_dir=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/participant 23 January 02, 2024.pickle')


In [ ]:
loaded_bf23_dif

In [ ]:
count_errorPT_TW(loaded_bf23_dir['NM'],22)

In [ ]:
loaded_bf23_dir

In [ ]:
lowest_error=100
best_tw=0
for tw in range(2,68):
  err=count_errorPT_TW([1.0 , 1, 1,.355,tw],21)
  if err<lowest_error:
    best_tw=tw
    lowest_error=err


In [ ]:
lowest_error

In [ ]:
best_tw

In [ ]:
count_errorPT_TW([1,1,1,.35,61],21)

In [ ]:
PT_TW_better

In [ ]:
equals